In [247]:
import numpy as np
import pandas as pd
import json
import datetime
import time

import re
import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords
import string


from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Nithin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [248]:
df = pd.read_csv('US_youtube_trending_data.csv')

In [249]:
df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...


In [250]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268787 entries, 0 to 268786
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   video_id           268787 non-null  object
 1   title              268787 non-null  object
 2   publishedAt        268787 non-null  object
 3   channelId          268787 non-null  object
 4   channelTitle       268787 non-null  object
 5   categoryId         268787 non-null  int64 
 6   trending_date      268787 non-null  object
 7   tags               268787 non-null  object
 8   view_count         268787 non-null  int64 
 9   likes              268787 non-null  int64 
 10  dislikes           268787 non-null  int64 
 11  comment_count      268787 non-null  int64 
 12  thumbnail_link     268787 non-null  object
 13  comments_disabled  268787 non-null  bool  
 14  ratings_disabled   268787 non-null  bool  
 15  description        264238 non-null  object
dtypes: bool(2), int64(5)

In [251]:
df.drop(columns=['video_id','thumbnail_link'],inplace=True, errors='ignore')

In [252]:
with open("US_category_id.json") as f:
    categoryID = json.load(f)

categoryID = categoryID['items']
ID_to_Category = {int(item['id']): item['snippet']['title'] for item in categoryID}

df['category_title'] = df['categoryId'].map(ID_to_Category)

In [253]:
ID_to_Category

{1: 'Film & Animation',
 2: 'Autos & Vehicles',
 10: 'Music',
 15: 'Pets & Animals',
 17: 'Sports',
 18: 'Short Movies',
 19: 'Travel & Events',
 20: 'Gaming',
 21: 'Videoblogging',
 22: 'People & Blogs',
 23: 'Comedy',
 24: 'Entertainment',
 25: 'News & Politics',
 26: 'Howto & Style',
 27: 'Education',
 28: 'Science & Technology',
 29: 'Nonprofits & Activism',
 30: 'Movies',
 31: 'Anime/Animation',
 32: 'Action/Adventure',
 33: 'Classics',
 34: 'Comedy',
 35: 'Documentary',
 36: 'Drama',
 37: 'Family',
 38: 'Foreign',
 39: 'Horror',
 40: 'Sci-Fi/Fantasy',
 41: 'Thriller',
 42: 'Shorts',
 43: 'Shows',
 44: 'Trailers'}

In [254]:
ID_to_Category.keys()

dict_keys([1, 2, 10, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])

In [255]:
df.head()

,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description,category_title
0,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,People & Blogs
1,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,False,False,"While running her own modding shop, Ramya Pare...",Gaming
2,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,False,False,I left youtube for a month and this is what ha...,Entertainment
3,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...,Music
4,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,False,False,Transforming The LaBrant Family's empty white ...,Howto & Style


In [256]:
def clean_trending_date(date):
    y,m,d = date.split('T')[0].split('-')
    return datetime.date(int(y), int(m), int(d))

def clean_publish_time(time):
    y,m,d = time.split('T')[0].split('-')
    return datetime.date(int(y), int(m), int(d))

In [257]:
df['trending_date'] = df['trending_date'].apply(clean_trending_date)
df['publishedAt'] = df['publishedAt'].apply(clean_publish_time)

df.head(n=2)

,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description,category_title
0,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,People & Blogs
1,Apex Legends | Stories from the Outlands – “Th...,2020-08-11,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,False,False,"While running her own modding shop, Ramya Pare...",Gaming


In [258]:
df.isnull().sum()

title                   0
publishedAt             0
channelId               0
channelTitle            0
categoryId              0
trending_date           0
tags                    0
view_count              0
likes                   0
dislikes                0
comment_count           0
comments_disabled       0
ratings_disabled        0
description          4549
category_title          0
dtype: int64

In [259]:
df = df.dropna()

In [260]:
df.isnull().sum()

title                0
publishedAt          0
channelId            0
channelTitle         0
categoryId           0
trending_date        0
tags                 0
view_count           0
likes                0
dislikes             0
comment_count        0
comments_disabled    0
ratings_disabled     0
description          0
category_title       0
dtype: int64

In [261]:
# Ensure the date columns are in datetime format
df['trending_date'] = pd.to_datetime(df['trending_date'])
df['publishedAt'] = pd.to_datetime(df['publishedAt'])
df['video_age'] = (df['trending_date'] - df['publishedAt']).dt.days

In [262]:
df.head()

,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description,category_title,video_age
0,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,People & Blogs,1
1,Apex Legends | Stories from the Outlands – “Th...,2020-08-11,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,False,False,"While running her own modding shop, Ramya Pare...",Gaming,1
2,I left youtube for a month and THIS is what ha...,2020-08-11,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,False,False,I left youtube for a month and this is what ha...,Entertainment,1
3,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...,Music,1
4,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,False,False,Transforming The LaBrant Family's empty white ...,Howto & Style,1


In [263]:

stop_words = set(stopwords.words('english'))

important_words = {'official', 'video', 'new', 'trailer', 'teaser', 'release', 'exclusive'}

def clean_titles(title):
    tokens = title.lower().split()
    cleaned = []
    for token in tokens:
        token = token.translate(str.maketrans('', '', string.punctuation))
        if (token in important_words or token not in stop_words) and token.isalnum():
            cleaned.append(token)
    return ' '.join(cleaned)

df['title_cl'] = df['title'].apply(clean_titles)

df[['title', 'title_cl']].head()


,title,title_cl
0,I ASKED HER TO BE MY GIRLFRIEND...,asked girlfriend
1,Apex Legends | Stories from the Outlands – “Th...,apex legends stories outlands
2,I left youtube for a month and THIS is what ha...,left youtube month happened
3,XXL 2020 Freshman Class Revealed - Official An...,xxl 2020 freshman class revealed official anno...
4,Ultimate DIY Home Movie Theater for The LaBran...,ultimate diy home movie theater labrant family


In [264]:
df.head()

,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description,category_title,video_age,title_cl
0,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,People & Blogs,1,asked girlfriend
1,Apex Legends | Stories from the Outlands – “Th...,2020-08-11,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,False,False,"While running her own modding shop, Ramya Pare...",Gaming,1,apex legends stories outlands
2,I left youtube for a month and THIS is what ha...,2020-08-11,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,False,False,I left youtube for a month and this is what ha...,Entertainment,1,left youtube month happened
3,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...,Music,1,xxl 2020 freshman class revealed official anno...
4,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,False,False,Transforming The LaBrant Family's empty white ...,Howto & Style,1,ultimate diy home movie theater labrant family


In [265]:
df['title_cl'][0]

'asked girlfriend'

In [266]:
df.head(1)

,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description,category_title,video_age,title_cl
0,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,People & Blogs,1,asked girlfriend


In [267]:
df.describe()

,publishedAt,categoryId,trending_date,view_count,likes,dislikes,comment_count,video_age
count,264238,264238.000000,264238,2.642380e+05,2.642380e+05,264238.000000,2.642380e+05,264238.00000
mean,2022-06-10 04:08:12.537787648,18.714061,2022-06-14 08:05:22.618245376,2.722629e+06,1.306312e+05,1098.990637,1.024426e+04,4.16470
min,2020-08-03 00:00:00,1.000000,2020-08-12 00:00:00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.00000
25%,2021-07-01 00:00:00,17.000000,2021-07-05 00:00:00,4.726832e+05,1.789100e+04,0.000000,1.303000e+03,2.00000
50%,2022-06-14 00:00:00,20.000000,2022-06-17 00:00:00,9.358560e+05,3.993050e+04,0.000000,2.783000e+03,4.00000
75%,2023-05-12 00:00:00,24.000000,2023-05-16 00:00:00,2.102609e+06,9.775700e+04,463.000000,6.433000e+03,5.00000
max,2024-04-14 00:00:00,29.000000,2024-04-15 00:00:00,1.407644e+09,1.602153e+07,879354.000000,6.738537e+06,37.00000
std,NaN,6.798227,NaN,9.794649e+06,4.539077e+05,7937.852758,7.320983e+04,2.56187


In [268]:
df.corr()

ValueError: could not convert string to float: 'I ASKED HER TO BE MY GIRLFRIEND...'

In [269]:
# Export the processed dataframe to a CSV file
processed_file_path = 'processed_youtube_data.csv'
df.to_csv(processed_file_path, index=False)

In [270]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import explode, split, col
from pyspark.ml.feature import StopWordsRemover, Tokenizer, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.sql.types import StringType
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
import time


In [271]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("YouTube Trending Videos Analysis") \
    .getOrCreate()

In [272]:
# Load cleaned dataset
data = spark.read.csv('processed_youtube_data.csv', header=True, inferSchema=True)

In [273]:
data = data.filter(col("title_cl").isNotNull() & (col("title_cl") != ""))

In [274]:
# Check for null values in any column
null_count = data.filter(data["title_cl"].isNull() | (col("title_cl") == "")).count()

if null_count > 0:
    print(f"There are {null_count} rows with null or empty values in 'title_cl'.")
else:
    print("No null or empty values found in 'title_cl'.")

No null or empty values found in 'title_cl'.


In [275]:
data = data.dropna()

In [276]:
words_df = data.select(split(col("title_cl"), " ").alias("words"))

In [277]:
words_df.show(5)

+--------------------+
|               words|
+--------------------+
|[apex, legends, s...|
|[left, youtube, m...|
|[ultimate, diy, h...|
|[havent, honest, ...|
|[first, family, i...|
+--------------------+
only showing top 5 rows



In [278]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
pipeline = Pipeline(stages=[remover])
cleaned_words_df = pipeline.fit(words_df).transform(words_df)

In [279]:
cleaned_words_df.show(5)

+--------------------+--------------------+
|               words|      filtered_words|
+--------------------+--------------------+
|[apex, legends, s...|[apex, legends, s...|
|[left, youtube, m...|[left, youtube, m...|
|[ultimate, diy, h...|[ultimate, diy, h...|
|[havent, honest, ...|[havent, honest, ...|
|[first, family, i...|[first, family, i...|
+--------------------+--------------------+
only showing top 5 rows



In [280]:
exploded_df = cleaned_words_df.select(explode(col("filtered_words")).alias("word"))

In [281]:
exploded_df.show(20)

+--------+
|    word|
+--------+
|    apex|
| legends|
| stories|
|outlands|
|    left|
| youtube|
|   month|
|happened|
|ultimate|
|     diy|
|    home|
|   movie|
| theater|
| labrant|
|  family|
|  havent|
|  honest|
|  injury|
|   heres|
|   truth|
+--------+
only showing top 20 rows



In [282]:
top_words = exploded_df.limit(20).collect()

In [283]:
# Print the collected words
for row in top_words:
    print(row['word'])

apex
legends
stories
outlands
left
youtube
month
happened
ultimate
diy
home
movie
theater
labrant
family
havent
honest
injury
heres
truth


In [284]:
# Machine Learning Analysis for Category Prediction
# Select relevant columns and drop rows with null values in these columns
model_data = data.select("title_cl", "category_title").dropna()

In [285]:
# Split the data into training and test sets
train_data, test_data = model_data.randomSplit([0.8, 0.2], seed=42)

In [286]:
# Tokenize the titles
tokenizer = Tokenizer(inputCol="title_cl", outputCol="words")

In [287]:
# Compute term frequencies
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)

In [288]:
# Compute the IDF (Inverse Document Frequency)
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [289]:
# Index the labels (categories)
indexer = StringIndexer(inputCol="category_title", outputCol="label")

In [290]:
# Define the Naive Bayes classifier
nb = NaiveBayes(modelType="multinomial")

In [291]:
# Create a pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, indexer, nb])

In [292]:
# Train the model
model = pipeline.fit(train_data)

In [293]:
# Make predictions on the test data
predictions = model.transform(test_data)

In [294]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test set accuracy = {accuracy:.2f}")

24/06/25 18:07:47 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


Test set accuracy = 0.74


In [295]:
# Select specific columns to print (adjust column names as needed)
#predictions.select("features", "label", "prediction").show()

# Alternatively, collect specific columns and print them
#predictions_list = predictions.select("features", "label", "prediction").collect()
#for row in predictions_list:
   # print(f"Features: {row['features']}, Label: {row['label']}, Prediction: {row['prediction']}")

In [296]:
###RandomnForrest in Pyspark

In [297]:
###PySpark and Time Capture####

In [298]:
# Data preprocessing (if necessary)
data = data.withColumn("view_count", col("view_count").cast("double"))
data = data.withColumn("likes", col("likes").cast("double"))
data = data.withColumn("categoryId", col("categoryId").cast("double"))

In [299]:
# Handle null values
data = data.na.fill(0, subset=["view_count", "likes", "categoryId"])


In [300]:
#StringIndexer for the label column
label_indexer = StringIndexer(inputCol="category_title", outputCol="label").fit(data)

In [301]:
# Assemble features into a feature vector
assembler = VectorAssembler(inputCols=["view_count", "likes", "categoryId"], outputCol="features")

In [302]:
# Split the data into training and test sets
train_data2, test_data2 = data.randomSplit([0.8, 0.2], seed=1234)

In [303]:
# Initialize the RandomForestClassifier
ps_rf_classifier = RandomForestClassifier(labelCol="label", featuresCol="features")

In [304]:
# Create a pipeline
pipeline = Pipeline(stages=[label_indexer, assembler, ps_rf_classifier])

In [305]:
# Start the timer
start_time = time.time()

In [306]:
# Train the model
model2 = pipeline.fit(train_data2)

In [307]:
# Make predictions
predictions2 = model2.transform(test_data2)

In [308]:
# Evaluate the model
evaluator2 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy2 = evaluator2.evaluate(predictions2)
accuracy2

0.9889917135426117

In [309]:
# Stop the timer
Execution_time = time.time() - start_time
print(f"Training Time: {training_time} seconds")

Training Time: 34.73774695396423 seconds


In [310]:
###Python and Time Capture####

In [311]:
####RandomForrest Classfier in Spark#####

In [312]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, roc_curve, auc


In [313]:
df.head(5)

,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description,category_title,video_age,title_cl
0,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,People & Blogs,1,asked girlfriend
1,Apex Legends | Stories from the Outlands – “Th...,2020-08-11,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,False,False,"While running her own modding shop, Ramya Pare...",Gaming,1,apex legends stories outlands
2,I left youtube for a month and THIS is what ha...,2020-08-11,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,False,False,I left youtube for a month and this is what ha...,Entertainment,1,left youtube month happened
3,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...,Music,1,xxl 2020 freshman class revealed official anno...
4,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,False,False,Transforming The LaBrant Family's empty white ...,Howto & Style,1,ultimate diy home movie theater labrant family


In [314]:
# Selecting relevant columns for the model
features = ["view_count", "likes", "categoryId"]
target = 'category_title' #Predicting video category

In [315]:
X = df[features]
y = df[target]

In [316]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [317]:
rf_classifier = RandomForestClassifier(random_state=42)

In [318]:
#Code to captutre time

start_time = time.time()
rf_classifier.fit(X_train, y_train)
end_time = time.time()


In [319]:
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [320]:
predictions = rf_classifier.predict(X_test)


In [321]:
accuracy = accuracy_score(y_test, predictions)
accuracy

0.999735089312746

In [322]:
# Record the time taken by the model to run
execution_time = end_time - start_time
print("Execution Time:", execution_time)

Execution Time: 20.990861892700195
